In [1]:
import os
import yaml
import torch
from torch.nn import DataParallel
from llama import Llama, Dialog

# Assuming you're running JupyterLab on a machine with 2 GPUs
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12356'  # Choose any free port number
os.environ['RANK'] = "0"
os.environ['WORLD_SIZE'] = "2"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import yaml
from llama import Llama, Dialog

# Define model and tokenizer paths
ckpt_dir = '/network/weights/llama.var/llama2/llama-2-13b-chat/'
tokenizer_path = '/network/weights/llama.var/llama2/tokenizer.model'

# Model parameters
max_seq_len = 512
max_batch_size = 8
temperature = 0.6
top_p = 0.9
max_gen_len = None  # Set to your preference

In [3]:
! echo $MASTER_ADDR
! echo $MASTER_PORT
! echo $RANK
! echo $WORLD_SIZE
! echo $CUDA_VISIBLE_DEVICES

localhost
12356
0
2
0,1


In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
%%time
# Initialize the Llama model
generator = Llama.build(
    ckpt_dir=ckpt_dir,
    tokenizer_path=tokenizer_path,
    max_seq_len=max_seq_len,
    max_batch_size=max_batch_size,
)

print("Model loaded!")

In [ ]:
# Load dialogs (adapt this to your dialog format)
dialogs = yaml.load(open('../data/prompts/example.yaml'), Loader=yaml.FullLoader)
print("Running inference...")

In [ ]:
%%time
# Generate responses
results = generator.chat_completion(
    dialogs,
    max_gen_len=max_gen_len,
    temperature=temperature,
    top_p=top_p,
)

In [ ]:
# Process and print results
for dialog, result in zip(dialogs, results):
    for msg in dialog:
        print(f"{msg['role'].capitalize()}: {msg['content']}\n")
    print(
        f"> {result['generation']['role'].capitalize()}: {result['generation']['content']}"
    )
    print("\n==================================\n")